Code from this script : 

https://www.kaggle.com/jeffd23/the-nature-conservancy-fisheries-monitoring/deep-learning-in-the-deep-blue-lb-1-279/notebook

In [2]:
# import numpy,pandas,matplotlib
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
# import scikit learn libraries

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split



In [4]:
# import keras libraries
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,Dropout,Flatten,MaxPooling2D,ZeroPadding2D,Activation
from keras.optimizers import Adam,RMSprop
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.preprocessing import image
from keras import backend as K

Using Theano backend.


In [5]:
TRAIN_DIR = 'train/train/'
TEST_DIR = 'test_stg1/test_stg1/'
FISH_CLASSES = ["ALB","BET","DOL","LAG","NoF","OTHER","SHARK","YFT"]
ROWS = 90
COLS = 160
CHANNELS = 3

In [1]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1]

def get_images(fish):
    """Load files from train folder"""
    fish_dir = TRAIN_DIR+ '{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir+'/')]
    return images

def read_image(img_path):
    """Read and resize individual images"""
    img = image.load_img(img_path,target_size=(90,160))
    x = image.img_to_array(img)
    x = vgg_preprocess(x)
    return x


files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

NameError: name 'np' is not defined

In [ ]:
X_all = np.ndarray((len(files), CHANNELS,ROWS, COLS), dtype=np.uint8)

for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

In [ ]:
y_all = LabelEncoder().fit_transform(y_all)
y_all = np_utils.to_categorical(y_all)

X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y_all)

In [ ]:
optimizer = RMSprop(lr=1e-4)
objective = 'categorical_crossentropy'


model = VGG16(weights = "imagenet")





In [ ]:
model.compile(loss=objective, optimizer = "sgd")

In [ ]:
# Original fit function

early_stopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto')        
        
model.fit(X_train, y_train, batch_size=64, nb_epoch=1,
              validation_split=0.2, verbose=1, shuffle=True, callbacks=[early_stopping])


In [14]:
model.fit(X_train, y_train, batch_size=20, nb_epoch=1,
              validation_split=0.2, verbose=1, shuffle=True)

Train on 2416 samples, validate on 605 samples
Epoch 1/1
2416/2416 [==============================] - 1066s - loss: 2.0223 - val_loss: 1.9712


In [15]:
preds = model.predict(X_valid, verbose=1)
print("Validation Log Loss: {}".format(log_loss(y_valid, preds)))

756/756 [==============================] - 106s   
Validation Log Loss: 1.9626615209869607


In [18]:
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)

1000/1000 [==============================] - 145s   


In [19]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.63992,0.512145,0.436537,0.426624,0.516825,0.449915,0.464633,0.521914
1,img_00007.jpg,0.63992,0.512145,0.436537,0.426624,0.516825,0.449915,0.464633,0.521914
2,img_00009.jpg,0.63992,0.512145,0.436537,0.426624,0.516825,0.449915,0.464633,0.521914
3,img_00018.jpg,0.63992,0.512145,0.436537,0.426624,0.516825,0.449915,0.464633,0.521914
4,img_00027.jpg,0.63992,0.512145,0.436537,0.426624,0.516825,0.449915,0.464633,0.521914


In [21]:
submission.to_csv("test_cnn1.csv",index = False)

Score : 1.96619


In [25]:
print(np.tile("ALB",3))

['ALB' 'ALB' 'ALB']


In [3]:
import cv2
from keras.preprocessing import image

In [13]:
rows = 90
cols = 160

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (cols, rows), interpolation=cv2.INTER_CUBIC)
    return im

In [18]:
img_path = "other/lion.jpg"

In [15]:
cv_version = read_image(img_path)

In [16]:
cv_version.shape

(90, 160, 3)

In [22]:
keras_version = image.img_to_array(image.load_img(img_path,target_size=(rows,cols)))

In [23]:
keras_version.shape

(3, 90, 160)